# 식품의약품안전처_의료기기 포장단위정보 (식의약헬스종합정보)
---
- 작성자 : 안지은
- 실행일 : 2023-01-10

In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

In [2]:
# 총 데이터 수 구하는 함수
def get_total_cnt(key):
    url = 'https://apis.data.go.kr/1471000/MdeqPcknUnitInfoService02/getMdeqPcknUnitInq02?serviceKey=%s&type=json'%(key)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['body']['totalCount'])
    return cnt

In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def get_api_conts(key,page):
    
    url = 'https://apis.data.go.kr/1471000/MdeqPcknUnitInfoService02/getMdeqPcknUnitInq02?serviceKey=%s&pageNo=%s&numOfRows=100&type=json'%(key,str(page))
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['body']['items']
    
    return data

### API 가져오기
---
- 한 페이지 당  100개 씩 제한
- 총 102430개 행

In [5]:
pkgs = []
cnt = get_total_cnt(key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts(key,i)
    for item in js:
        pkgs.append(item)


100%|██████████| 1025/1025 [11:33<00:00,  1.48it/s]


### 데이터 프레임 변환
---
- 컬럼명 : API 목록

In [6]:
pkgs_df = pd.json_normalize(pkgs)
pkgs_df.columns = ['의료기기품목일련번호','기관지역구분','업종','허가신고구분','의료기기품목허가번호','품목명',
'허가일자','취소취하구분','취소취하일자','상품명','일련번호','포장단위명']
pkgs_df

,의료기기품목일련번호,기관지역구분,업종,허가신고구분,의료기기품목허가번호,품목명,허가일자,취소취하구분,취소취하일자,상품명,일련번호,포장단위명
0,106,서울청,제조업,신고,서울 제신 06-182 호,체외형범용프로브,2006-11-10,None,None,None,1,자사 포장단위에 의함
1,108,서울청,제조업,신고,서울 제신 06-183 호,의료용프로브(체외형),2006-11-10,None,None,None,1,자사 포장단위에 의함
2,110,서울청,수입업,신고,서울 수신 06-1679 호,의료용가이드,2006-11-10,None,None,None,1,제조원의 포장단위에 의함.
3,114,서울청,제조업,신고,서울 제신 06-181 호,의료용절삭기구,2006-11-10,None,None,Bonesier,1,자사 포장단위에 의함
4,1200200209,경인청,제조업,신고,경인 제신 03-25 호,부목,2003-03-07,None,None,None,1,1개
...,...,...,...,...,...,...,...,...,...,...,...,...
102425,70,부산청,수입업,신고,부산 수신 06-40 호,시력보정용안경렌즈,2006-11-07,None,None,None,1,1매 또는 1개
102426,72,부산청,수입업,신고,부산 수신 06-38 호,시력보정용안경렌즈,2006-11-07,None,None,None,1,1매 또는 1개
102427,74,부산청,수입업,신고,부산 수신 06-37 호,시력보정용안경렌즈,2006-11-07,None,None,None,1,1매 또는 1개
102428,76,의료기기정보기술지원센터,수입업,인증,수인 06-1144 호,근전도계,2006-11-07,None,None,NIM-Neuro 2.0 System,1,1UNIT/BOX
